In [ ]:
from mido import Message
import pretty_midi
import numpy as np
import pretty_midi
from sample_synth import sample_synthesis

sample_rate = 44100

def get_notes_from_track(track):
    notes = []
    for note in track:
        notes.append({
            'pitch': note.note,
            'start': note.start,
            'end': note.end,
            'velocity': note.velocity,
            'synth': []
        })
    return notes

def synthesis (track):
    track_notes = get_notes_from_track(track)
    
    min_pitch = min(note['pitch'] for note in track_notes)
    max_pitch = max(note['pitch'] for note in track_notes)
    max_end = max(note['end'] for note in track_notes)

    for note in track_notes:
        note['synth'] = sample_synthesis(note['pitch'],  note['velocity'], note['end'] - note['start'])

    num_pitches = max_pitch - min_pitch + 1
    num_samples = int(np.ceil(max_end))*sample_rate
    tracks_s = np.zeros((num_pitches, num_samples))

    for note in track_notes:
        pitch = note['pitch'] - min_pitch
        start_frame = round(note['start']*sample_rate)
        # end_frame = int(np.ceil(note['end']*sample_rate))
        end_frame = start_frame + len(note['synth'])
        tracks_s[pitch, start_frame:end_frame] = note['synth']

    return np.sum(tracks_s, axis=0)


# samples = synthesis(data)

In [ ]:
def procesar_midi_messages(midi_messages):
    int_tiempos = {}
    int_velocities = {}
    
    for message in midi_messages:
        pitch = message.pitch
        start = message.start
        end = message.end
        velocity = message.velocity
        
        if pitch not in int_tiempos:
            int_tiempos[pitch] = []
            int_velocities[pitch] = []
        
        # Añadir el intervalo de tiempo
        int_tiempos[pitch].append([start, end])
        
        # Añadir la velocidad para el intervalo de tiempo
        int_velocities[pitch].append(velocity)
    
    return int_tiempos, int_velocities

In [ ]:
            
mid_data_ins = pretty_midi.PrettyMIDI('midi_files/test_file.mid')
# track = mid_data_ins.instruments[node_box[i-1]]
            